In [97]:
import os
import json
import boto3

import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

from dotenv import load_dotenv
from plotly.subplots import make_subplots
from sqlalchemy import create_engine, text

load_dotenv()

ACCESS_KEY = os.getenv("ID_KEY_ACCESS")
ACCESS_KEY_SECRETE = os.getenv("KEY_SECRET")
BUCKET_NAME = os.getenv("BUCKETNAME")

Python-dotenv could not parse statement starting at line 5


## Récupération des données récolté

In [98]:
df_1 = pd.read_json("./hotel_1.json")
df_2 = pd.read_json("./hotel_2.json")
df_3 = pd.read_json("./temps.json")

In [99]:
# merge des 3 dataframe
def merge_outer(data_1:pd.DataFrame, data_2:pd.DataFrame, on:str) -> pd.DataFrame:
    df = pd.merge(data_1, data_2, how='outer', on=on)
    return df

df = merge_outer(df_1, df_2, "hotel")
df = merge_outer(df, df_3, "id")
df.dropna()

In [45]:
# Sauvegarde du dataframe en csv pour envoie dans un s3
csv = df.to_csv()
df.to_csv("hotel.csv")

## Stockage des données en S3

In [6]:
# Création d'une session aws
session = boto3.Session(aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=ACCESS_KEY_SECRETE)
s3 = session.resource("s3")

In [7]:
# Création d'un bucket s3
bucket = s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [8]:
# Ajout du fichier csv dans le compartiment bucket s3
put_object = bucket.put_object(Key="hotel.csv", Body=csv)

## Récupération des données

In [9]:
# recuperation du fichier csv dans un s3
df = pd.read_csv(f's3://{BUCKET_NAME}/hotel.csv')

In [101]:
# Modifiquation des données pour mise en SQL
df_temps = df[["id", "ville", "lat", "long", "temperature", "temps"]]

df_temps.drop_duplicates("id", inplace=True)
df_temps.reset_index(drop=True, inplace=True)

df_temps["moyenne"] = df_temps["temperature"].apply(lambda x : round((sum(x) / len(x)),2) )

jour = list(range(1,6))
df_temps['jour'] = [jour for _ in range(len(df_temps))]

df_temps = df_temps.apply(pd.Series.explode)

df["note"] = df["note"].str.replace(",", ".")
df["note"] = pd.to_numeric(df["note"])

df.drop(["lat", "long", "temperature", "temps"], inplace=True, axis=1)

C:\Users\jerem\AppData\Local\Temp\ipykernel_24592\2603906745.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jerem\AppData\Local\Temp\ipykernel_24592\2603906745.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jerem\AppData\Local\Temp\ipykernel_24592\2603906745.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

## Base SQL

In [104]:
# Création de la base SQL
# engine = create_engine(f"postgresql+psycopg2://{YOUR_USERNAME}:{YOUR_PASSWORD}@{YOUR_HOSTNAME}/postgres", echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)

In [105]:
df.to_sql("hotel", engine)

2023-01-04 08:22:14,144 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hotel")
2023-01-04 08:22:14,145 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:22:14,146 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hotel")
2023-01-04 08:22:14,146 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:22:14,147 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:22:14,148 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotel (
	"index" BIGINT, 
	id BIGINT, 
	ville TEXT, 
	hotel TEXT, 
	url TEXT, 
	note FLOAT, 
	description TEXT, 
	latitutde FLOAT, 
	longitude FLOAT
)


2023-01-04 08:22:14,149 INFO sqlalchemy.engine.Engine [no key 0.00061s] ()
2023-01-04 08:22:14,150 INFO sqlalchemy.engine.Engine CREATE INDEX ix_hotel_index ON hotel ("index")
2023-01-04 08:22:14,150 INFO sqlalchemy.engine.Engine [no key 0.00037s] ()
2023-01-04 08:22:14,151 INFO sqlalchemy.engine.Engine COMMIT
2023-01-04 08:22:14,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:22:14

875

In [106]:
df_temps.to_sql("temps", engine)

2023-01-04 08:22:21,962 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("temps")
2023-01-04 08:22:21,963 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:22:21,964 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("temps")
2023-01-04 08:22:21,965 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-04 08:22:21,967 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:22:21,968 INFO sqlalchemy.engine.Engine 
CREATE TABLE temps (
	"index" BIGINT, 
	id BIGINT, 
	ville TEXT, 
	lat FLOAT, 
	long FLOAT, 
	temperature FLOAT, 
	temps TEXT, 
	moyenne FLOAT, 
	jour BIGINT
)


2023-01-04 08:22:21,969 INFO sqlalchemy.engine.Engine [no key 0.00092s] ()
2023-01-04 08:22:21,970 INFO sqlalchemy.engine.Engine CREATE INDEX ix_temps_index ON temps ("index")
2023-01-04 08:22:21,971 INFO sqlalchemy.engine.Engine [no key 0.00048s] ()
2023-01-04 08:22:21,972 INFO sqlalchemy.engine.Engine COMMIT
2023-01-04 08:22:21,974 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-04 08:22:21,977

175

In [107]:
# Connexion à la base
conn = engine.connect()

In [108]:
# Requête pour récupérer les 5 meilleurs villes
ville = text("SELECT * FROM temps ORDER BY moyenne DESC LIMIT 25")

In [109]:
# Requête pour récupérer les hôtels
hotel_1 = text("SELECT * FROM hotel WHERE id = 21 ORDER BY note DESC LIMIT 20")
hotel_2 = text("SELECT * FROM hotel WHERE id = 20 ORDER BY note DESC LIMIT 20")
hotel_3 = text("SELECT * FROM hotel WHERE id = 35 ORDER BY note DESC LIMIT 20")
hotel_4 = text("SELECT * FROM hotel WHERE id = 2 ORDER BY note DESC LIMIT 20")
hotel_5 = text("SELECT * FROM hotel WHERE id = 1 ORDER BY note DESC LIMIT 20")

In [110]:
# Récupération sous forme de dataframe pour visualisation
hotel_1 = pd.read_sql_query(hotel_1, conn)
hotel_2 = pd.read_sql_query(hotel_2, conn)
hotel_3 = pd.read_sql_query(hotel_3, conn)
hotel_4 = pd.read_sql_query(hotel_4, conn)
hotel_5 = pd.read_sql_query(hotel_5, conn)
hotel_viz = pd.concat([hotel_1,hotel_2,hotel_3,hotel_4,hotel_5], ignore_index=False)
ville_viz = pd.read_sql_query(ville, conn)

2023-01-04 08:22:29,400 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 21 ORDER BY note DESC LIMIT 20
2023-01-04 08:22:29,401 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ()
2023-01-04 08:22:29,404 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 20 ORDER BY note DESC LIMIT 20
2023-01-04 08:22:29,405 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()
2023-01-04 08:22:29,407 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 35 ORDER BY note DESC LIMIT 20
2023-01-04 08:22:29,408 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()
2023-01-04 08:22:29,410 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 2 ORDER BY note DESC LIMIT 20
2023-01-04 08:22:29,411 INFO sqlalchemy.engine.Engine [generated in 0.00056s] ()
2023-01-04 08:22:29,413 INFO sqlalchemy.engine.Engine SELECT * FROM hotel WHERE id = 1 ORDER BY note DESC LIMIT 20
2023-01-04 08:22:29,413 INFO sqlalchemy.engine.Engine [generated in 0.00055s] ()
2023-01-04 08:22:

## Visualisation

In [111]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(
    ville_viz,
    lat='lat',
    lon='long',
    color='temperature',
    size='temperature',
    color_continuous_scale=px.colors.diverging.Picnic,
    size_max=30,
    zoom=4.7,
    hover_name='ville',
    hover_data={
        'lat': False,
        'long': False,
        'moyenne': True        
        },
    animation_frame='jour'
)

fig.update_layout(width = 1500, height = 800, template='plotly_dark' ,title='Les 5 meilleurs ville sur un moyenne de température')
fig.show()

In [93]:
hotel_viz

,index,id,ville,hotel,url,note,description,latitutde,longitude
0,511,21,"Toulouse Sud-Est, Toulouse","La loge du Minotaure, cosy, parking et bureaux",https://www.booking.com/hotel/fr/59m2-parking-...,9.6,Doté d'une aire de jeux pour enfants et d'une ...,43.575154,1.473728
1,500,21,"Toulouse Rive Gauche, Toulouse",LA CARTOUCHERIE,https://www.booking.com/hotel/fr/la-cartoucher...,9.4,Vous pouvez bénéficier d'une réduction Genius ...,43.601721,1.408088
2,509,21,"Centre de Toulouse, Toulouse",Les Loges des Chalets,https://www.booking.com/hotel/fr/les-loges-des...,9.2,"Situé à 5.1 km du Stadium de Toulouse, l'établ...",43.614051,1.447940
3,514,21,"Minimes - Barrière de Paris, Toulouse",Large and modern flat just nearby Ponts-Jumeau...,https://www.booking.com/hotel/fr/large-and-mod...,8.9,Le Large and modern flat just nearby Ponts-Jum...,43.613013,1.421539
4,503,21,"Centre de Toulouse, Toulouse",Le Bonrepos - Appartement 2 chambres clim & pa...,https://www.booking.com/hotel/fr/appartement-d...,8.7,Vous pouvez bénéficier d'une réduction Genius ...,43.609439,1.453500
...,...,...,...,...,...,...,...,...,...
15,10,1,Le Mont-Saint-Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,7.9,Vous pouvez bénéficier d'une réduction Genius ...,48.616263,-1.510906
16,5,1,Le Mont-Saint-Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,7.8,Vous pouvez bénéficier d'une réduction Genius ...,48.617587,-1.510396
17,13,1,Ardevon,Auberge de la Baie,https://www.booking.com/hotel/fr/auberge-de-la...,7.8,Vous pouvez bénéficier d'une réduction Genius ...,48.615993,-1.488253
18,12,1,Le Mont-Saint-Michel,Hôtel la Croix Blanche,https://www.booking.com/hotel/fr/ha-el-la-croi...,7.6,Situé au cœur du village médiéval du Mont-Sain...,48.635734,-1.509861


In [113]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(
    hotel_viz,
    lat='latitutde',
    lon='longitude',
    color='note',
    size='note',
    color_continuous_scale=px.colors.diverging.Picnic,
    size_max=30,
    zoom=4.7,
    hover_name='ville',
    hover_data={
        'latitutde': False,
        'longitude': False,
        'hotel': True,
        'note': True    
        }, 
)

fig.update_layout(width = 1500, height = 800, template='plotly_dark' ,title='Les 5 meilleurs ville sur un moyenne de température')
fig.show()